In [1]:
import speech_recognition as sr
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import asyncio
import pytesseract

r = sr.Recognizer()
app = FastAPI()

origins = [
    "http://localhost:3000",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # Cho phép các nguồn gốc được xác định
    allow_credentials=True,
    allow_methods=["*"],  # Cho phép tất cả các phương thức (GET, POST, PUT, DELETE)
    allow_headers=["*"],  # Cho phép tất cả các headers
)

@app.get("/")
async def read_root():
    return {"Hello": "World"}

@app.post("/convertAudio")
async def convert_audio_to_text(file: UploadFile = File(...)):
    with open(f"./audio/{file.filename.split('.')[0]}.wav", "wb") as audio:
        audio.write(file.file.read())
    with sr.AudioFile("./audio/" + file.filename.split('.')[0] + ".wav") as source:
        audio = r.record(source)
    try:
        text = r.recognize_google(audio)
    except sr.UnknownValueError:
        text = "Could not understand the audio"
    except sr.RequestError as e:
        text = "Could not request results; {0}".format(e)
    return {"text": text}

@app.post("/getImg")
async def get_img(file: UploadFile = File(...)):
    with open(f"./img/{file.filename.split('.')[0]}.jpg", "wb") as img:
        img.write(file.file.read())
    return True

@app.get("/convertImg")
async def convert_img_to_text():
    text = pytesseract.image_to_string("img.jpg")
    return {"text": text}
        
if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [10736]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:61036 - "POST /convertAudio HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [10736]
